# **Data Preprocessing**

In [1]:
from datasets import load_dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql")
dataset

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})

In [2]:
import pandas as pd
df = pd.DataFrame(dataset["train"])
df.columns

Index(['id', 'domain', 'domain_description', 'sql_complexity',
       'sql_complexity_description', 'sql_task_type',
       'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql',
       'sql_explanation'],
      dtype='object')

In [3]:
def chat_template_for_training(context, answer, question):
    template = f"""\
    <|im_start|>user
    Given the context, generate an SQL query for the following question
    context:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant
    {answer}
    <|im_end|>
    """
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

In [4]:
df["text"] = df.apply(lambda x: chat_template_for_training(x["sql_context"], x["sql"], x["sql_prompt"]), axis=1)
print(df.head(1))
print(df.columns)

     id  ...                                               text
0  5097  ...  <|im_start|>user\nGiven the context, generate ...

[1 rows x 12 columns]
Index(['id', 'domain', 'domain_description', 'sql_complexity',
       'sql_complexity_description', 'sql_task_type',
       'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql',
       'sql_explanation', 'text'],
      dtype='object')


In [5]:
df = df.drop(['id', 'domain','domain_description','sql_complexity','sql_complexity_description','sql_task_type','sql_task_type_description','sql_prompt','sql_context','sql','sql_explanation'], axis=1)

In [6]:
df.columns

Index(['text'], dtype='object')

In [7]:
df.to_csv("train_data.csv",index=False)

In [1]:
import pandas as pd
df = pd.read_csv("train_data.csv")
len(df)

100000

In [2]:
df = df.head(20000)

In [3]:
len(df)

20000

In [4]:
df.to_csv("custum_data.csv",index=False)

In [1]:
import pandas as pd
df = pd.read_csv("new.csv")
df.columns

Index(['context', 'question', 'answer'], dtype='object')

In [2]:
def chat_template_for_training(context, answer, question):
    template = f"""\
    <|im_start|>user
    Given the context, generate an SQL query for the following question
    context:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant
    {answer}
    <|im_end|>
    """
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

In [3]:
df["text"] = df.apply(lambda x: chat_template_for_training(x["context"], x["answer"], x["question"]), axis=1)
print(df.head(1))
print(df.columns)

                                             context  \
0  create table details(acre integer(10), current...   

                                            question  \
0  What are the different types of soil present i...   

                                    answer  \
0  SELECT DISTINCT soil_type FROM details;   

                                                text  
0  <|im_start|>user\nGiven the context, generate ...  
Index(['context', 'question', 'answer', 'text'], dtype='object')


In [4]:
df = df.drop(['context','question','answer'], axis=1)
df.columns

Index(['text'], dtype='object')

In [5]:
df.to_csv("cust.csv",index=False)

In [11]:
import pandas as pd
df1 = pd.read_csv("cust.csv")
df2 = pd.read_csv("custum_data.csv")
concatenated_df = pd.concat([df1, df2], ignore_index=True)

In [14]:
concatenated_df.to_csv("final_dataset.csv",index=False)
len(concatenated_df)

20120

# **Training**

In [1]:
import pandas as pd
from datasets import load_dataset , Dataset

df = pd.read_csv("final_dataset.csv")
formatted_data = Dataset.from_pandas(df)
formatted_data

Dataset({
    features: ['text'],
    num_rows: 20120
})

In [2]:
from transformers import AutoTokenizer , AutoModelForCausalLM , BitsAndBytesConfig , TrainingArguments
from peft import LoraConfig , get_peft_model, prepare_model_for_kbit_training
from accelerate import Accelerator
from trl import SFTTrainer
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

In [3]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout = 0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model,lora_config)

In [4]:
training_args = TrainingArguments(
    output_dir = "Tiny-Llama-AgriDB-FineTuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate = 2e-4,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    save_steps=1000,
    logging_steps=500,
    num_train_epochs=2,
    fp16=True,
    push_to_hub=False
)

In [5]:
trainer = SFTTrainer(
    model=model,
    train_dataset = formatted_data,
    dataset_text_field="text",
    peft_config=lora_config,
    args=training_args,
    tokenizer = tokenizer,
    packing=False,
    max_seq_length=2056
)
trainer.train()

Map:   0%|          | 0/20120 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddharthmagesh007 (velammal-edu-in). Use `wandb login --relogin` to force relogin


  0%|          | 0/10060 [00:00<?, ?it/s]

d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
d:\newenv\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.5113, 'grad_norm': 0.796561062335968, 'learning_rate': 0.00019878344637963306, 'epoch': 0.1}
{'loss': 0.4462, 'grad_norm': 0.6546969413757324, 'learning_rate': 0.0001951633855727567, 'epoch': 0.2}


...

d:\newenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.3825, 'grad_norm': 0.5852525234222412, 'learning_rate': 1.5415995110600657e-06, 'epoch': 1.89}
{'loss': 0.385, 'grad_norm': 0.6110746264457703, 'learning_rate': 1.9352669449190785e-08, 'epoch': 1.99}


d:\newenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 13856.4342, 'train_samples_per_second': 2.904, 'train_steps_per_second': 0.726, 'train_loss': 0.40646289632050225, 'epoch': 2.0}


TrainOutput(global_step=10060, training_loss=0.40646289632050225, metrics={'train_runtime': 13856.4342, 'train_samples_per_second': 2.904, 'train_steps_per_second': 0.726, 'total_flos': 7.47976192758743e+16, 'train_loss': 0.40646289632050225, 'epoch': 2.0})

In [4]:
from huggingface_hub import login
login(token="")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Siddharth\.cache\huggingface\token
Login successful


In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True
)
fine_tuned_path = r"D:\agribot\Tiny-Llama-AgriDB-FineTuned\checkpoint-10000/"
peft_model= PeftModel.from_pretrained(model,fine_tuned_path,from_transformers=True,device_map="auto")

model = peft_model.merge_and_unload()
model.push_to_hub("siddharth-magesh/Tiny_Lllama-AgriDB")

model.safetensors:   0%|          | 0.00/2.41G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/siddharth-magesh/Tiny_Lllama-AgriDB/commit/90ef9f674bd5e3bbc108c30d7cb11c5d69247b84', commit_message='Upload LlamaForCausalLM', commit_description='', oid='90ef9f674bd5e3bbc108c30d7cb11c5d69247b84', pr_url=None, pr_revision=None, pr_num=None)

In [5]:
def chat_template(question, context):
    template = f"""\
    <|im_start|>user
    Given the context, generate an SQL query for the following question
    context:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant 
    """
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

In [6]:
#inference on GPU
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "siddharth-magesh/Tiny_Lllama-AgriDB"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")  
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token

# Prepare the Prompt
question = "Which animal type has the highest quantity recorded?"
context = "create table animals_details(cattlename varchar(20), quantity integer(5));"
prompt = chat_template(question,context)  # Assuming chat_template concatenates the strings.

# Encode the prompt
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Ensure inputs are on CPU

# Generate the output
output = model.generate(**inputs, max_new_tokens=512)

# Decode the output
text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated SQL query
print(text)

model.safetensors:   0%|          | 0.00/2.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

d:\newenv\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


<|im_start|>user
Given the context, generate an SQL query for the following question
context:create table animals_details(cattlename varchar(20), quantity integer(5));
question:Which animal type has the highest quantity recorded?
<|im_end|>
<|im_start|>assistant 
SELECT cattlename, MAX(quantity) as max_quantity FROM animals_details GROUP BY cattlename;
<|im_end|>
<|im_start|>assistant 
SELECT cattlename, quantity, MAX(quantity) as max_quantity FROM animals_details GROUP BY cattlename;
<|im_end|>
<|im_start|>assistant 
SELECT cattlename, quantity, MAX(quantity) as max_quantity FROM animals_details GROUP BY cattlename ORDER BY max_quantity DESC;
<|im_end|>
<|im_start|>assistant 
SELECT cattlename, quantity, MAX(quantity) as max_quantity FROM animals_details GROUP BY cattlename ORDER BY max_quantity DESC;
<|im_end|>
<|im_start|>assistant 
SELECT cattlename, quantity, MAX(quantity) as max_quantity FROM animals_details GROUP BY cattlename ORDER BY max_quantity DESC;
<|im_end|>
<|im_start|>a